# An inner parallel cutting plane method for convex mixed-integer optimization problems 

## Summary of the document

This document is complementary to the paper "Granularity in mixed-integer nonlinear optimization". The intention is to make the numerical results transparent. The generated results for the IPCP, the feasibility pump as well as for B-Hyb are stored as pandas data-frames and are concatenated herein. 

The latex-tables of the article are produced by using the command print(dataframe.to_latex(float_format = '%.2f'))

In [11]:
import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format
import numpy as np
import pickle
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

## First quick check

In [12]:
result = load_obj('res_overview_small_testset')

In [15]:
result

,vars,constrs,time L,time SOR,obj,constr_value,time_bonmin,obj_bonmin
cvxnonsep_normcon20r,"(40, 10, 0)","(21, 20, 10, 10)",$1.73,$0.08,$-14.65,$-inf,$0.21,$-15.26
cvxnonsep_normcon30r,"(60, 15, 0)","(31, 30, 15, 15)",$2.66,$0.07,$-14.80,$-inf,$0.16,$-22.71
ex4,"(36, 25, 25)","(30, 25, 25, 0)",$0.00,$0.06,$-6.70,$-inf,$0.98,$-6.86
ex1223a,"(7, 4, 4)","(9, 4, 4, 0)",$0.00,$0.06,$6.07,$-inf,$0.17,$4.58
ringpack_10_1,"(70, 50, 50)","(385, 330, 330, 0)",$0.00,$0.47,$-4.17,$-inf,$2.65,$-4.17
smallinvDAXr1b150-165,"(31, 30, 0)","(4, 1, 1, 1)",$0.20,$0.05,$224.69,$-124.03,$14.89,$164.97
